In [1]:
import custom_utils as cu
from ultralytics import YOLO
import numpy as np

import cv2

labels = ["green", "red", "yellow"]

In [2]:
# prepare environment
import os
from dotenv import load_dotenv
# Load .env files
load_dotenv()

# Get environment variables
dataset_dir = os.getenv('DATASET_DIR')
ori_data_dir = os.path.join(dataset_dir, "traffic_light", "original_data")
custom_data_dir = os.path.join(dataset_dir, "traffic_light", "custom_data")

if not os.path.exists(ori_data_dir):
    print("Error: No original data set")

if not os.path.exists(custom_data_dir):
    os.mkdir(custom_data_dir)

for l in labels:
    label_dir = os.path.join(custom_data_dir, l)
    if not os.path.exists(label_dir):
        os.mkdir(label_dir)

In [3]:
# step 0: get images 
img_dic = {}
for l in labels: 
    p, images = cu.get_images(os.path.join(ori_data_dir, l), 4)
    img_dic.update({l: images })

In [4]:
# step 1: analyze images with yolo
model = YOLO("yolov8n.pt")
res_dict = {}
for l in labels:
    print(l)
    res = model.predict(img_dic[l], save=True, save_txt = True)
    print(type(res[0]))
    res_dict.update({l : res})

green



0: 640x640 1 traffic light, 1: 640x640 2 traffic lights, 2: 640x640 1 traffic light, 3: 640x640 2 traffic lights, 399.5ms
Speed: 4.2ms preprocess, 99.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict14
4 labels saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict14/labels



<class 'ultralytics.engine.results.Results'>
red


0: 640x640 1 traffic light, 1: 640x640 (no detections), 2: 640x640 2 traffic lights, 3: 640x640 2 traffic lights, 398.6ms
Speed: 2.9ms preprocess, 99.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict14
4 labels saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict14/labels



<class 'ultralytics.engine.results.Results'>
yellow


0: 640x640 3 traffic lights, 3 apples, 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 1 person, 3 cars, 427.9ms
Speed: 3.3ms preprocess, 107.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict14
4 labels saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict14/labels


<class 'ultralytics.engine.results.Results'>


In [5]:
# todo 
# drop similar boxes is requiered becaus some times yolo identify the same object with a different bounding box
"""
def drop_similar_boxes(input_list, threshold):
    def are_elements_similar(elem1, elem2):
        x1_diff = abs(elem1[0] - elem2[0])
        y1_diff = abs(elem1[1] - elem2[1])
        x2_diff = abs(elem1[2] - elem2[2])
        y2_diff = abs(elem1[3] - elem2[3])
        return x1_diff < threshold or y1_diff < threshold or x2_diff < threshold or y2_diff < threshold

    unique_elements = []
    for elem in input_list:
        is_similar = False
        for unique_elem in unique_elements:
            if are_elements_similar(elem, unique_elem):
                is_similar = True
                break
        if not is_similar:
            unique_elements.append(elem)
    return unique_elements
"""
"""
input_list = [[1.0, 2.0, 3.0, 4.0], [1.2, 2.2, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0], [1.1, 2.1, 3.1, 4.1]]
threshold = 0.2
filtered_list = drop_similar_boxes(input_list, threshold)
print(filtered_list)
"""

'\ninput_list = [[1.0, 2.0, 3.0, 4.0], [1.2, 2.2, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0], [1.1, 2.1, 3.1, 4.1]]\nthreshold = 0.2\nfiltered_list = drop_similar_boxes(input_list, threshold)\nprint(filtered_list)\n'

In [6]:
def average_resize(label, images):
    print("type of images: ")
    print(type(images[0]))
    dia_len = [(i.shape[1]**2 + i.shape[2])**0.5 for i in images]
    avr = int(np.average(dia_len))
    for idx, i in enumerate(images):
        print("hi")
        i = cv2.resize(i, (avr,avr))
        #cv2.imwrite(os.path.join(custom_data_dir, label, str(idx), ".jpg"), i) # not nessecary that label is in the name of image

In [7]:
print(type(res_dict["red"]))

<class 'list'>


In [8]:
# step 2: extract all traffic lights from all analyzed images and create for each an new image

# Process results list
image_name = 'traffic_light_green'
file_type = '.jpg'

counter = 0
extracted_dic = {}
for l in labels:
    print(l)
    ex_boxes = [] # is an list because probably there are more traffic_lights objects in a picture
    for j, result in enumerate(res_dict[l]):
        for i,c in enumerate(result.boxes.cls.numpy()):
            # find traffic light boxes

            points = result.boxes.xyxyn.numpy()[i]
            if c == 9:
                #path = out_path + image_name + str(counter) + file_type
                extracted = cu.extract_rectangle_from_image(images[j], points) 
                ex_boxes.append(extracted)
                res_dict[l][j] = extracted # extracted.append(new_image)
                print(type(res_dict[l][j]))
                counter += 1
        extracted_dic.update({l : ex_boxes})
            
    # resize imgaes
    average_resize(l, extracted_dic[l])
    out_path = os.path.join(custom_data_dir, l)
    print(out_path)
    cu.write_images(extracted_dic[l], out_path, main_name=l)

green
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
type of images: 
<class 'numpy.ndarray'>
hi
hi
hi
hi
hi
hi
/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/dataset/traffic_light/custom_data/green
red
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
type of images: 
<class 'numpy.ndarray'>
hi
hi
hi
hi
hi
/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/dataset/traffic_light/custom_data/red
yellow
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
type of images: 
<class 'numpy.ndarray'>
hi
hi
hi
/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/dataset/traffic_light/custom_data/yellow


# step 3: check images manually and label them